In [ ]:
import pandas as pd
from fbprophet import Prophet

# Raw data
model_df = pd.read_csv('https://raw.githubusercontent.com/Data-Science-Squad/dssquad-ml/dm-model/prophet-demo/model_data.csv')

# Group data by Geo, Geo_Level, and Freq
grouped = model_df.groupby(["Geo", "Geo_Level", "Freq"])

# Simple Prophet function to fit/forecast
def prophet_function(train, test):
    m = Prophet()
    forecast = m.fit(train).predict(test)
    out = forecast[['yhat', 'yhat_lower', 'yhat_upper']]
    return(out)

# Loop through groups, apply prophet_function, and append output to list
models = []
for name, group in grouped:
    train = group[(group['model_date'] >= '2015-01-01') & (group['model_date'] <= '2019-12-31')]
    test = group[(group['model_date'] >= '2020-01-01') & (group['model_date'] <= '2020-12-31')]

    prophet_train = train.rename(columns={"model_date": "ds", 'Incidents': "y"})
    prophet_test = test.rename(columns={"model_date": "ds", 'Incidents': "y"})  

    prophet_fcast = prophet_function(prophet_train, prophet_test)

    test = test.reset_index(drop=True)
    prophet_fcast = prophet_fcast.reset_index(drop=True)
    test_with_fcast = test.join(prophet_fcast)

    out = pd.concat([train, test_with_fcast])

    models.append(out)

# Concatenate list of models
models_df = pd.concat(models)

models_df.head()